In [1]:
from selenium import webdriver # 셀레니움과 웹드라이버 부르는 모듈함수 
import time
import pymysql

   
    
# croll 2번씩 내려감
def scroll_dowmn_page2():
    for i in range(2): # adjust integer value for need
        driver.execute_script("window.scrollBy(0, 200)")
        time.sleep(0.5)
    
    

# law-talk inquirty 데이터 추출
def inquirty_data():
    inquirty = []
    segment = []

    Org = 4 # Org = 4는, law-talk-after_1days 즉,1일 이후에 작성된 게시글을 모으기위한 용도
    try :
        time.sleep(2)
        # segment1~3 주제 위에 있는 소분류 예시: 가압류/가처분건설/부동산민사소송절차
        # segment 1개 있 때, 
        frist_segment = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/h5/span").text
          
        
        # segment가 3개이상일 때,   
        thrid_segment = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/h5/span[1]").text
        
        if thrid_segment == frist_segment :
            segment.append(thrid_segment)
    
        thrid_segment2 = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/h5/span[2]").text
        thrid_segment3 = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/h5/span[3]").text
        segment.append(thrid_segment2)
        segment.append(thrid_segment3)


        # segment가 비어있다면, 스페이스바를 넣어야함
        # 이유는 db에 넣을 때, 인덱스 번호를 맞춰주기 위함
        if len(frist_segment) == 0:
            segment.append(" ")
            
        if len(thrid_segment) == 0:
            segment.append(" ")

        if len(thrid_segment2) == 0:
            segment.append(" ")

        if len(thrid_segment3) == 0:
            segment.append(" ")
            
    
    except :
        pass
    
    Ttile = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/h1").text
    content = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/p").text
    content = content.replace("\n","")
 
    regdata = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/div[2]/span").text
    
    
    
    Tag = "";
    # ★★★★★ Tag 가 비어있을 때, 인식을 못함, if문으로 ★★★★★
    try :
        Tag = driver.find_element_by_xpath("//*[@id='root-view']/div/div/div/question/div/div[1]/recommend-keyword/div").text
        Tag = Tag.split("\n")
        del Tag[0] # 관련 키워드 삭제   

    except Exception as e :
        print("관련 키워드가 비어 있어서 except 처리")
        print(e)
        print("\n")
        Tag = "" # ★★★★★ 여기서 Tag를 꼭 reset해줘야 해당 Tag들이 제대로 저장 됨 ★★★★★
        pass
        
        
        
    inquirty.append(Org)
    inquirty.append(segment)
    inquirty.append(Ttile)
    inquirty.append(content)
    
    
    # regdate (게시글 작성 시간)을 기준하여, 1일 이후 일 때만, 데이터를 추출한다.
    # 즉, 게시글 작성 시간 기준 "분" , "시간"을 제외한 나머지를 추출함(1일 이후 데이터들)
    if "분" in regdata :
        pass
    elif "시간" in regdata :
        pass
    else :
        inquirty.append(regdata)

    inquirty.append(Tag)
    
    
    
    
    
    # ★★★★★★★ 해당 링크페이지 및 사건번호 추출 ★★★★★★★
    elems = driver.find_elements_by_xpath("//link[@href]")  # 페이지 클릭 후, 해당 페이지에서 link 태그에서 href 의 부분 추출

    arr = []
    for elem in elems:
        arr.append(elem.get_attribute("href"))


    link = arr[0].split("-")[0]   # https://www.lawtalk.co.kr/qna/179309
    inquirty.append(link)

    num = arr[0].split("/")[4].split("-")[0]   # 179117
    inquirty.append(num)
    
    insert_inquiry(inquirty) # aws - db commit
    return inquirty

    
    
    
# AWS DB 연동
def get_db():
    db = pymysql.connect(
        host='ec2-13-125-144-25.ap-northeast-2.compute.amazonaws.com',
        port=3306,
        user='admin',
        passwd='1/zw;GytAwx*', 
        db='laws',
        charset='utf8'
    )
    return db






# AWS DB에 데이터 넣기 
def insert_inquiry(data):
    db = get_db()
    mycursor = db.cursor()

    try:
        sql = "INSERT INTO Inquiry (Num,OrgIndex,Segment,Title,content,regDate,Tag,Link) VALUES (%s, %s, %s, %s, %s, %s, %s,%s)"
        val = (data[7],data[0],delete_mystr(str(data[1])),str(data[2]),str(data[3]),str(data[4]),delete_mystr(str(data[5])),data[6])

        mycursor.execute(sql, val)

        db.commit()
        db.close()
    
    
    except Exception as e:
        print("1일 이전에 작성된 게시글은 저장하지 않기 때문에,regdate 인덱스가 비어 있음.")
        print("그렇기 때문에, list index out of range 에러가 발생함")
        print("즉, 해당 에러는 1일 이전에 작성된 게시글을 저장X")
        print('db insert inquiry error', e)
        print("\n")
        db.close()


# "[","]" delete function 
def delete_mystr(str):
    str = str.replace("[","")
    str = str.replace("]","")
    return str


      
    
    
# ------------ main --------------
driver = webdriver.Chrome('C:\/chromedriver.exe') # 제어하기 위해 변수 선언
driver.maximize_window() ### 창의 해상도 크기 최대화 (전체화면) ###

    
    
try :    
    time.sleep(2)
    scroll_dowmn_page2()
    
    try :

            # 1page 1~1611 까지 페이지 이동
            for i in range(10,1612):
                keyword = '사기'
                url = f'https://www.lawtalk.co.kr/cases?pg={i}&sort=recentAnswer&keyword={keyword}'
                driver.get(url)
                
                # 한 페이지에 있는 게시글 10개 데이터들을 추출
                for i in range(1,11):
                    time.sleep(4)
                    case_board = driver.find_element_by_xpath(f"//*[@id='root-view']/div[2]/div[3]/section/div/div[4]/div/section/case-card[{i}]/a/div")
                    case_board.click()
                    
                    
                    inquirty_data() # law-talk-inquiry data extract & db data commit &  close
                    driver.back() # 뒤로가기
                    time.sleep(5)
 
    except Exception as e :
        print("게시글 클릭 error",e)
    
except Exception as e :
    print("데이터 로딩 error",e)

관련 키워드가 비어 있어서 except 처리
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='root-view']/div/div/div/question/div/div[1]/recommend-keyword/div"}
  (Session info: chrome=91.0.4472.124)



관련 키워드가 비어 있어서 except 처리
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='root-view']/div/div/div/question/div/div[1]/recommend-keyword/div"}
  (Session info: chrome=91.0.4472.124)



게시글 클릭 error Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='root-view']/div[2]/div[3]/section/div/div[4]/div/section/case-card[1]/a/div"}
  (Session info: chrome=91.0.4472.124)

